In [3]:
import boto3
import io
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import awswrangler as wr

import warnings
warnings.filterwarnings("ignore")

plt.style.use('seaborn-colorblind')

### set-up

In [4]:
session = boto3.Session()
s3 = session.client('s3')

In [5]:
bucket_name = "cdo-ililapse-364524684987-bucket"
file_path = "x266754/lapse/"

### data intake

In [6]:
file_name = "x266754/lapse/curated_test.parquet"

In [7]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
df = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 38.6 s, sys: 14.3 s, total: 52.9 s
Wall time: 29.7 s


In [8]:
df[df['policy_id']=='V9063643'][['policy_id', 'month', 
                                 'lapse_ind', 
                                 'surrender_ind', 
                                 'reinstate_ind', 
                                 'current_status',
                                 '3mo_ahead_status']]

,policy_id,month,lapse_ind,surrender_ind,reinstate_ind,current_status,3mo_ahead_status
4951461,V9063643,1,0.0,0.0,0.0,current,current
4951462,V9063643,2,0.0,0.0,0.0,current,current
4951463,V9063643,3,0.0,0.0,0.0,current,current
4951464,V9063643,4,0.0,1.0,0.0,current,surrender


### Target Variable

In [9]:
df['3mo_ahead_Lapse'] = (np.where((df['3mo_ahead_status']=='lapse') | ((df['3mo_ahead_status']=='surrender')), 1,0))
df['current_lapse_status'] = (np.where((df['current_status']=='lapse') | ((df['current_status']=='surrender')), 1,0))

### Features

In [10]:
# policy holder
df['holder_age'] = df['attained_age']

In [11]:
# policy age
df['policy_age'] = np.round((df['val_dt'] -df['issue_dt'].astype('datetime64[ns]'))/np.timedelta64(1, 'M'), 0).astype(int)

In [12]:
# issue year
df['issue_year']=  df['issue_dt'].astype('datetime64[ns]').dt.year

In [13]:
# cash value
df['mod_mpt_total'] = df['mpt_tot_act_val'] - df['tot_loan_prncpl']
df['mod_mpt3'] = df['mpt_3'] - df['tot_loan_prncpl']
df['mod_mpt4'] = df['mpt_4'] - df['tot_loan_prncpl']

In [14]:
# payment
df['mod_bld_payment'] = df['modal_prem_bld'] + df['unpd_loan_int_due']
df['mod_sch_payment'] = df['modal_prem_sch'] + df['unpd_loan_int_due']

In [15]:
# cash value - payment
df['cash_accum_prem'] = df['mpt_tot_act_val'] - df['accum_actual_prem_paid']
df['diff_mod_bld_value'] = df['mod_mpt_total'] - df['mod_bld_payment']

In [16]:
# normalized
df['mpt_total_bld_face_amt'] = df['mod_mpt_total']/df['base_face_amt']

In [ ]:
# month over month change in cash value
df['mod_mpt_total_lag1'] = df.groupby(['policy_id'])['mod_mpt_total'].shift(1)
df['mod_mpt_total_lag1'] = np.where(df['mod_mpt_total_lag1'].isnull(), df['mod_mpt_total'], df['mod_mpt_total_lag1'])
df['mom_mod_mpt_total']= df['mod_mpt_total'] - df['mod_mpt_total_lag1'] 

In [17]:
# month over month change in payment 
df['accum_actual_prem_paid_lag1'] = df.groupby(['policy_id'])['accum_actual_prem_paid'].shift(1)
df['accum_actual_prem_paid_lag1'] = np.where(df['accum_actual_prem_paid_lag1'].isnull(), df['accum_actual_prem_paid'], df['accum_actual_prem_paid_lag1'])
df['mom_accum_prem_paid']= df['accum_actual_prem_paid'] - df['accum_actual_prem_paid_lag1'] 

In [18]:
# month over month change in prem mode
df['prem_mode_lag1'] = df.groupby(['policy_id'])['prem_mode'].shift(1)
df['prem_mode_lag1'] = np.where(df['prem_mode_lag1'].isnull(), df['prem_mode'], df['prem_mode_lag1'])
df['prem_mode_change_flag']= np.where((df['prem_mode_lag1'] !=df['prem_mode']), 1,0)

In [19]:
# household AUM
df['diff_household_asset_cash_val'] = df['wc_tot_asset_amt']  - df['mod_mpt_total']
df['hh_stock_mix'] = df['wc_asset_mix_stk_amt']/df['wc_tot_asset_amt'] 
df['hh_bond_mix'] = df['wc_asset_mix_bond_amt']/df['wc_tot_asset_amt'] 
df['hh_annuity_mix'] = df['wc_asset_mix_anty_amt']/df['wc_tot_asset_amt'] 

In [20]:
# subject aum
df['diff_aum_cash_val'] = df['subj_aum_amt']  - df['mod_mpt_total']

In [21]:
# other mpt
df['rest_mpt'] = df['mpt_10']+df['mpt_600']+df['mpt_620']+df['mpt_1600']+df['mpt_20881']

In [22]:
# rider flag
df['rdr_face_amt'] = df['rdr1_face_amt']+df['rdr2_face_amt']+df['rdr3_face_amt']
df['rider_flag'] =np.where(df['rdr_face_amt']>0, 1, 0)

In [23]:
# interaction
df['holder_age_cash']=df['holder_age']*df['mod_mpt_total']
df['policy_age_cash']=df['policy_age']/12*df['mod_mpt_total']
df['issue_age_cash']=df['issue_age']*df['mod_mpt_total']

df['diff_policy_issue_issue']=df['policy_age_cash']-df['issue_age_cash']
df['diff_cash_holder_issue']=df['holder_age_cash']-df['issue_age_cash']

In [24]:
# rolling count
df['current'] = df['current_status'].map(lambda x: 1 if x == 'current' else 0)
df['lapse'] = df['current_status'].map(lambda x: 1 if x == 'lapse' else 0)
df['reinstate'] = df['current_status'].map(lambda x: 1 if x == 'reinstated' else 0)

In [25]:
%%time
df['num_current_3m'] = df.groupby('policy_id')['current'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
df['num_lapse_3m'] = df.groupby('policy_id')['lapse'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
df['num_reinstate_3m'] = df.groupby('policy_id')['reinstate'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)

#df['num_current_3m'] = df.groupby('policy_id')['current'].transform(lambda x: x.rolling(3).sum())
# df['num_lapse_3m'] = df.groupby('policy_id')['lapse'].transform(lambda x: x.rolling(3).sum())
# df['num_reinstate_3m'] = df.groupby('policy_id')['reinstate'].transform(lambda x: x.rolling(3).sum())

CPU times: user 1min 44s, sys: 2.15 s, total: 1min 46s
Wall time: 1min 46s


In [26]:
# backward fill
df['num_current_3m']=df['num_current_3m'].bfill()
df['num_reinstate_3m']=df['num_reinstate_3m'].bfill()
df['num_lapse_3m']=df['num_lapse_3m'].bfill()

In [27]:
# number of policies per household and month over month change
count_df = pd.DataFrame(df[(df['household_id']!=888888888)].groupby(['household_id','val_dt'])['agmt_pkge_id'].count()).reset_index()
count_df.columns = ['household_id', 'val_dt', 'num_policies_hh']

In [28]:
%%time
df = df.merge(count_df, how= 'left', 
                      left_on =['household_id','val_dt'],
                      right_on =['household_id','val_dt']
                     )

CPU times: user 3min 52s, sys: 1min 59s, total: 5min 52s
Wall time: 5min 51s


In [29]:
# month over month change in number of policies per household
df['num_policies_hh_lag1'] = df.groupby(['policy_id'])['num_policies_hh'].shift(1)
df['num_policies_hh_lag1'] = np.where(df['num_policies_hh_lag1'].isnull(), df['num_policies_hh'], df['num_policies_hh_lag1'])
df['mom_hh_policies']= df['num_policies_hh'] - df['num_policies_hh_lag1'] 

In [30]:
# number of service requests last 6 months
df['num_sr_catg'] = df['num_sr_catg'].fillna(0)
df['max_sr_time'] = df['max_sr_time'].fillna(0)
df['time_to_close'] = df['time_to_close'].fillna(0)

df['num_sr_6m'] = df.groupby('policy_id')['num_sr_catg'].rolling(window=6, min_periods=1).sum().reset_index(level=0, drop=True)
df['num_sr_6m']=df['num_sr_6m'].bfill()

In [31]:
# current month - last service request month 
# sr_month_df = pd.DataFrame(df[df['num_sr_catg']>=1].groupby(['policy_id'])['pfmc_cur_month'].last()).reset_index()
# sr_month_df.columns = ['policy_id', 'last_sr_month']

In [32]:
# %%time
# df = df.merge(sr_month_df, how= 'left', 
#                       left_on =['policy_id','pfmc_cur_month'],
#                       right_on =['policy_id','last_sr_month']
#                      )

### buckets 

In [33]:
#age bucket
col         = 'holder_age'  
conditions  = [(df[col] <= 20), (df[col] >20) & (df[col] <= 30),  
               (df[col] >30) & (df[col] <= 35),  (df[col] >35) & (df[col] <= 40), 
               (df[col] >40) & (df[col] <= 45), (df[col] >45) & (df[col] <= 50),  
               (df[col] >50) & (df[col] <= 55), (df[col] >55) & (df[col] <= 60), 
               (df[col] >60) & (df[col] <= 65), (df[col] >65) & (df[col] <= 70), 
               (df[col] >70) & (df[col] <= 75), (df[col] >75) & (df[col] <= 80), 
               (df[col] >80) & (df[col] <= 85), df[col] >85]  
              
choices     = [ '1. <20', '2. (20,30]', '3. (30,35]', '4. (35,40]', '5. (40,45]', '6. (45,50]',
               '7. (50,55]', '8. (55,60]', '9. (60,65]', '10. (65,70]', '11. (70,75]','12. (75,80]', 
               '13. (80,85]','14. >85']
    
df["holder_age_bucket"] = np.select(conditions, choices, default='missing')

In [34]:
#cash value
col         = 'mod_mpt_total'  
conditions  = [(df[col] >=0) & (df[col] <= 10000),
               (df[col] >10000) & (df[col] <= 25000),
               (df[col] >25000) & (df[col] <= 50000),
               (df[col] >50000) & (df[col] <= 100000),  
               (df[col] >100000) & (df[col] <= 250000),  
               (df[col] >250000) & (df[col] <= 500000),  
               (df[col] >500000) & (df[col] <= 750000), 
               (df[col] >750000) & (df[col] <= 1000000),
               (df[col] >1000000) & (df[col] <= 3000000),
               (df[col] >3000000) & (df[col] <= 5000000), 
               (df[col] > 5000000)]  
              
choices     = [ '1. $0k-10k', '2. $10k-25k', '3. $25k-50k','4. $50k-100k', 
               '5. $100k-250k', '6. $250k-500k','7. $500k-750k', '8. $750k-1M', 
               '9. $1M-3M', '10. $3M-5M','11. >$5M']
    
df["cash_val_bucket"] = np.select(conditions, choices, default='missing')

In [35]:
pd.set_option('display.max_columns', None)
df[df['policy_id']=='V9063643'][['policy_id', 'household_id', 
                                 'month', 
                                 'num_policies_hh', 'mom_hh_policies',
                                 'holder_age',  "holder_age_bucket",
                                 'issue_year', 'policy_age',
                                 'mod_mpt_total', 'cash_val_bucket',
                                 'mod_bld_payment', 
                                 'cash_accum_prem', 'diff_mod_bld_value',
                                 'mpt_total_bld_face_amt',  
                                 'accum_actual_prem_paid', 'mom_accum_prem_paid',
                                 'prem_mode', 'prem_mode_change_flag', 
                                 'diff_household_asset_cash_val', 
                                 'hh_stock_mix', 'hh_bond_mix', 'hh_annuity_mix',
                                 'diff_aum_cash_val', 
                                 'rest_mpt',
                                 'rider_flag',
                                 'diff_policy_issue_issue',
                                 'diff_cash_holder_issue',
                                 'num_current_3m',
                                 'num_lapse_3m',
                                 'num_reinstate_3m',
                                 'num_sr_catg',
                                 'num_sr_6m',
                                # 'last_sr_month',
                                # 'month_since_last_sr',
                                 '3mo_ahead_status']]

,policy_id,household_id,month,num_policies_hh,mom_hh_policies,holder_age,holder_age_bucket,issue_year,policy_age,mod_mpt_total,cash_val_bucket,mod_bld_payment,cash_accum_prem,diff_mod_bld_value,mpt_total_bld_face_amt,accum_actual_prem_paid,mom_accum_prem_paid,prem_mode,prem_mode_change_flag,diff_household_asset_cash_val,hh_stock_mix,hh_bond_mix,hh_annuity_mix,diff_aum_cash_val,rest_mpt,rider_flag,diff_policy_issue_issue,diff_cash_holder_issue,num_current_3m,num_lapse_3m,num_reinstate_3m,num_sr_catg,num_sr_6m,3mo_ahead_status
4951461,V9063643,26921591.0,1,1.0,0.0,25.0,"2. (20,30]",2007,177,2359.36,1. $0k-10k,30.0,-2027.0,2329.36,0.039323,5310.0,0.0,P,0,399660.64,0.222596,0.003069,0.00239,847.47,3283.0,0,8847.600000,33031.04,1.0,0.0,0.0,0.0,0.0,current
4951462,V9063643,26921591.0,2,1.0,0.0,25.0,"2. (20,30]",2007,178,2339.36,1. $0k-10k,30.0,-2077.0,2309.36,0.038989,5340.0,30.0,P,0,399680.64,0.222596,0.003069,0.00239,890.00,3263.0,0,8967.546667,32751.04,2.0,0.0,0.0,0.0,0.0,current
4951463,V9063643,26921591.0,3,1.0,0.0,25.0,"2. (20,30]",2007,179,2373.36,1. $0k-10k,30.0,-2073.0,2343.36,0.039556,5370.0,30.0,P,0,399646.64,0.222596,0.003069,0.00239,918.68,3297.0,0,9295.660000,33227.04,3.0,0.0,0.0,0.0,0.0,current
4951464,V9063643,26921591.0,4,1.0,0.0,25.0,"2. (20,30]",2007,180,2229.36,1. $0k-10k,30.0,-2247.0,2199.36,0.037156,5400.0,30.0,P,0,399790.64,0.222596,0.003069,0.00239,968.69,3153.0,0,8917.440000,31211.04,3.0,0.0,0.0,0.0,0.0,surrender


### Data Filters

In [36]:
# remove decseased  (2 = deceased)
#df= df[df['deceasead_ind']!=2]

# remove policies with zero base face amount 
df = df[df['base_face_amt']!=0]

# remove negative policies with negative policy age
df = df[df['policy_age']>=0]

### export features 

In [37]:
!pip install awswrangler --q

Keyring is skipped due to an exception: 'keyring.backends'


In [38]:
%%time
import awswrangler as wr

wr.s3.to_parquet(
    df=df,
    path='s3://cdo-ililapse-364524684987-bucket/x266754/lapse/curated_features.parquet'
)

CPU times: user 1min 47s, sys: 20.5 s, total: 2min 7s
Wall time: 1min 42s


{'paths': ['s3://cdo-ililapse-364524684987-bucket/x266754/lapse/curated_features.parquet'],
 'partitions_values': {}}